In [1]:
# =========================================================
# Dynamic Optimization with SINDy (Guard-railed Formulation)
# - Direct transcription in STANDARDIZED space (Z,V)
# - RK4 with substeps for stability
# - Buffers scaled to [0,1]
# - Trust regions, move limits, smooth nonnegativity
# - Soft demand slack and regularization
# =========================================================

import numpy as np
import casadi as ca
import pandas as pd
from tqdm.auto import tqdm

# ---------------------------------------------------------
# CONFIG
# ---------------------------------------------------------

# Horizon & discretization
T_hours   = 100.0
Delta_t   = 1.0            # hours (choose so that delays are integer steps)
N = int(T_hours/Delta_t)   # number of steps

# Delays (hours) -> integer steps at this Delta_t
tau_AA_to_APAP  = 5
tau_APAP_to_H2  = 30.0
tau_PAP_to_APAP = 0.0

d_AA_to_APAP   = int(round(tau_AA_to_APAP/Delta_t))   # = 5 when Delta_t=1
d_APAP_to_H2   = int(round(tau_APAP_to_H2/Delta_t))   # = 30 when Delta_t=1
d_PAP_to_APAP  = int(round(tau_PAP_to_APAP/Delta_t))  # = 0

# Buffers: capacities (kg) and initials (kg)
Bmax_AA_APAP   = 500000.0
B0_AA_APAP     = 400000.0

Bmax_PAP_APAP  = 600000.0
B0_PAP_APAP    = 500000.0

Bmax_APAP_H2   = 5000000000.0
B0_APAP_H2     = 4000000000.0

# Demand: APAP and H2 cumulative production over [0,T] (kg)
D_APAP = 10000
D_H2 = 50000

In [2]:
# ---------------------------------------------------------
# Per-plant dimensions (adjust to your real models)
# ---------------------------------------------------------
nx_AA,  nu_AA  = 5, 2
nx_PAP, nu_PAP = 5, 4
nx_APAP,nu_APAP= 4, 2
nx_H2,  nu_H2  = 5, 1   

# Indices for objective/arrivals (adjust if different)
#   NOTE: waste indices are indices of state components that represent waste RATES (kg/h)
WASTE_IDX_AA    = [2, 4]       # e.g. x1_AA, x2_AA
WASTE_IDX_PAP   = [0, 1, 4]    # e.g. PAP waste channels
WASTE_IDX_APAP  = [2]          # e.g. x1_APAP

#No Waste coming from Green Hydrogen as it is clean industry

# APAP production rate index (kg/h) used for demand integral
IDX_APAP_PROD = 0
IDX_H2_PROD = 3

# Arrival source indices (state components that feed the buffers)
IDX_AA_TO_APAP      = 0  # AA commodity rate feeding AA→APAP buffer
IDX_PAP_TO_APAP     = 2  # PAP commodity rate feeding PAP→APAP buffer
IDX_APAP_WASTE_TO_H2 = 1  # APAP circularizable waste feeding APAP→H2 buffer (edit if needed)

# Withdrawal mapping: which APAP controls correspond to pulling from AA/PAP buffers (kg/h)
# (Edit if your control semantics differ.)
U_APAP_WITHDRAW_IDX_AA  = 0
U_APAP_WITHDRAW_IDX_PAP = 1
U_H2_WITHDRAW_IDX_APAPWASTE = 0

# ---------------------------------------------------------
# ASSIGN THE ACTUAL ARRAY VALUES (Standardization scalers)
#   Set to your training scaler stats (μ and σ) for each plant.
#   Order must match the order used to train each SINDy model.
# ---------------------------------------------------------
mu_x_AA     = np.array([2.52633361e+04, 4.20439760e+03, 3.08744099e+04, 1.39513803e+00, 3.94209102e+04])
sigma_x_AA  = np.array([8.68867166e+00, 2.48127643e+01, 2.63294970e+02, 1.97747577e-02, 6.57181056e+01])
mu_u_AA     = np.array([5.00065296e+04, 5.07058504e+04])
sigma_u_AA  = np.array([7.11117642e+01, 4.17289885e+01])

mu_x_PAP    = np.array([49.94379286, 2.5999891, 9.58745948, 4.25643191, 9.26802494])
sigma_x_PAP = np.array([2.29394217, 0.46491546, 1.49445646, 0.16805903, 0.16901018])
mu_u_PAP    = np.array([35.00022868, 14.94508125, 26.94796864, 1.99783992])
sigma_u_PAP = np.array([1.66664278, 1.59267972, 1.48080084, 0.2637756])

mu_x_APAP    = np.array([3900.33182582, 8712.8850495, 3778.44232477, 3790.3929877])
sigma_x_APAP = np.array([12.63603063, 10.71029953, 23.0314057, 11.21707354])
mu_u_APAP    = np.array([2652.22628937, 3174.40414659])
sigma_u_APAP = np.array([10.65174245, 10.67409751])

mu_x_H2    = np.array([190027.82383972, 1991396.47429357, 1119471.77632037, 250910.42689571, 120216.54806859])
sigma_x_H2 = np.array([2864.81179763, 30021.04258232, 16876.77495219, 3782.56406699, 1812.35452077])
mu_u_H2    = np.array([3672035.23038146])
sigma_u_H2 = np.array([55358.68195529])

# Initial states (physical units). Replace with your measured values.
x0_AA   = np.array([25260.7, 4189.88, 31015.2, 1.40762, 39473.8])
x0_PAP  = np.array([49.7846, 2.53388, 7.45591, 3.92756, 9.59563])
x0_APAP = np.array([3925.08, 8689.31, 3740.82, 3817.91])
x0_H2  = np.array([192622.939, 2018598.81, 1134764.49, 254337.848, 121858.286])

# ---------------------------------------------------------
# SINDy RHS IN STANDARDIZED VARIABLES (dz/dt = fz(z,u))
#   Use the exact SINDy equations you printed (standardized space).
# ---------------------------------------------------------

def fz_AA(zx, zu):
    z0, z1, z2, z3, z4 = zx[0], zx[1], zx[2], zx[3], zx[4]
    u0, u1 = zu[0], zu[1]

    dx0 = ( 2.433 + (-3.196)*z0 + 12.865*z1 + 121.763*z2 + (-33.931)*z3 + 8.046*z4
            + (-77.567)*u0 + (-5.002)*z0*z0 + 81.444*z0*z1 + 57.289*z0*z2 + 18.908*z0*z3
            + (-3.049)*z0*u0 + 3.083*z0*u1 + (-48.868)*z1*z1 + (-11.813)*z1*u0 + 25.725*z2*z2
            + 50.098*z2*z3 + 2.807*z2*z4 + (-34.317)*z2*u0 + 1.638*z2*u1 + (-20.313)*z3*z3
            + 5.368*z3*u0 + 2.371*z3*u1 + 0.246*z4*z4 + 3.307*u0*u0 + (-2.783)*u0*u1 )

    dx1 = ( -1.997 + 8.135*z0 + (-137.922)*z1 + (-94.322)*z2 + (-2.301)*z3
            + (-36.360)*u0 + (-5.445)*u1 + (-1.807)*z0*z0 + 30.562*z0*z1 + 24.588*z0*z2
            + 4.302*z0*z3 + (-1.809)*z0*u0 + 1.315*z0*u1 + (-11.949)*z1*z1 + 10.430*z2*z2
            + 3.448*z2*z3 + (-2.384)*z2*u0 + 1.264*z2*u1 + 0.233*z3*u1 + 0.100*z4*z4
            + 0.715*u0*u0 + (-0.380)*u0*u1 )

    dx2 = ( -0.355 + (-3.194)*z0 + 42.618*z1 + (-0.001)*z2 + (-3.838)*z3
            + 45.654*u0 + (-0.007)*u1 + 2.445*z0*z0 + (-46.250)*z0*z1 + (-42.038)*z0*z2
            + (-3.295)*z0*z3 + 3.079*z0*u0 + (-2.417)*z0*u1 + 19.033*z1*z1 + (-19.547)*z2*z2
            + (-1.926)*z2*z3 + (-0.367)*z2*z4 + 4.655*z2*u0 + (-1.731)*z2*u1 + (-0.210)*z3*u1
            + (-0.132)*z4*z4 + 0.297*u0*u1 )

    dx3 = ( 1.615 + (-6.522)*z0 + 119.189*z1 + 98.295*z2 + (-9.162)*z3 + 0.096*z4
            + 27.776*u0 + 5.591*u1 + 0.603*z0*z0 + (-11.995)*z0*z1 + (-7.173)*z0*z2
            + (-3.958)*z0*z3 + 0.212*z0*u0 + (-0.344)*z0*u1 + 2.709*z1*z1 + (-0.982)*z2*z2
            + (-3.160)*z2*z3 + (-0.221)*z2*u1 + (-0.196)*z3*u1 + (-0.059)*z4*z4
            + 0.549*u0*u0 + 0.205*u0*u1 )

    dx4 = ( 0.931 + (-36.852)*z0 + (-36.591)*z1 + 28.509*z2 + 5.755*z3 + (-255.697)*z4
            + 23.315*u0 + 222.789*u1 + 1.168*z0*z0 + (-5.292)*z0*z1 + (-4.361)*z0*z3
            + 2.963*z0*z4 + 3.556*z1*z1 + (-3.326)*z1*z4 + 2.485*z1*u0 + (-5.401)*z1*u1
            + (-1.110)*z2*z2 + (-8.757)*z2*u1 + (-3.552)*z3*z3 + 1.871*z3*u0 + (-2.656)*z3*u1
            + (-0.669)*z4*z4 + 0.750*u0*u0 + 5.702*u0*u1 )

    return ca.vcat([dx0, dx1, dx2, dx3, dx4])

def fz_PAP(zx, zu):
    z0, z1, z2, z3, z4 = zx[0], zx[1], zx[2], zx[3], zx[4]
    u0, u1, u2, u3 = zu[0], zu[1], zu[2], zu[3]

    dx0 = ( 0.401 + (-245.618)*z0 + (-5.724)*z1 + (-0.074)*z2 + 0.356*z3
            + 177.581*u0 + 170.074*u1 + (-0.053)*u2 + 5.530*u3 + 4.087*z0*z2 + 0.870*z0*u3
            + (-1.505)*z1*z1 + 6.736*z1*z2 + (-6.901)*z1*z4 + (-0.496)*z1*u0 + 0.022*z2*z2
            + 0.070*z2*z4 + (-3.312)*z2*u0 + (-2.438)*z2*u1 + 0.232*z2*u2 + (-6.611)*z2*u3
            + 0.539*z3*u0 + (-0.453)*z3*u1 + 0.106*z3*u2 + (-6.334)*z3*u3 + (-0.044)*z4*z4
            + (-0.138)*u0*u0 + 0.086*u0*u1 + (-0.041)*u0*u2 + (-0.192)*u1*u1 + (-0.011)*u1*u2
            + (-0.422)*u1*u3 + 0.011*u2*u2 + 0.214*u2*u3 + 1.010*u3*u3 )

    dx1 = ( 1.896 + (-0.063)*z0 + (-298.786)*z1 + 0.340*z2 + (-14.500)*z4
            + (-0.214)*u0 + (-19.365)*u2 + 296.838*u3 + (-2.235)*z0*z0 + 1.884*z0*z1
            + 0.133*z0*z4 + 0.511*z0*u2 + (-2.857)*z1*z1 + (-180.344)*z1*z3 + (-186.130)*z1*z4
            + (-0.232)*z1*u0 + (-6.115)*z1*u2 + 0.139*z2*z2 + (-0.392)*z2*z4 + (-0.030)*z2*u0
            + (-0.332)*z2*u1 + (-0.091)*z2*u2 + 0.758*z2*u3 + (-90.723)*z3*z3 + 33.733*z3*u2
            + 96.107*z4*z4 + 0.343*z4*u1 + 34.905*z4*u2 + (-1.223)*z4*u3 + 0.657*u0*u0
            + 2.360*u0*u1 + (-0.100)*u0*u2 + (-1.111)*u0*u3 + 0.851*u1*u1 + (-1.310)*u1*u3
            + (-1.371)*u2*u2 + 11.667*u2*u3 + (-3.268)*u3*u3 )

    dx2 = ( -0.130 + 0.096*z0 + 0.355*z1 + (-0.244)*z2 + (-0.040)*z3
            + 0.032*u0 + 1.081*u2 + (-0.349)*u3 + (-0.066)*z0*u2 + 0.376*z1*z1
            + (-0.132)*z1*z4 + 0.217*z2*z3 + 0.154*z2*u0 + (-0.178)*z2*u1 + (-0.520)*z2*u2
            + (-0.111)*z2*u3 + 0.088*z3*z3 + 0.088*z3*u1 + (-0.132)*z3*u3 + 0.146*z4*u0
            + 0.393*z4*u2 + (-0.007)*u0*u0 + 0.052*u0*u1 + 0.096*u0*u3 + (-0.047)*u1*u1
            + (-0.067)*u1*u2 + (-0.030)*u1*u3 + 0.029*u2*u2 + 0.160*u2*u3 + (-0.336)*u3*u3 )

    dx3 = ( -0.033 + 0.002*z2 + (-0.001)*z3 + (-0.013)*z4 + (-0.001)*z2*z4 + 0.003*z4*z4 )

    dx4 = ( 0.032 + (-0.002)*z2 + (-0.012)*z3 + (-0.001)*z4 + 0.001*u2 + 0.002*z2*z4 + (-0.003)*z4*z4 )

    return ca.vcat([dx0, dx1, dx2, dx3, dx4])

def fz_APAP(zx, zu):
    z0, z1, z2, z3 = zx[0], zx[1], zx[2], zx[3]
    u0, u1 = zu[0], zu[1]

    dx0 = ( -1.076 + (-1.805)*z0 + (-69.013)*z1 + 13.201*z2 + (-56.065)*z3
            + (-0.686)*u0 + 8.687*u1 + 29.510*z0*z1 + (-2.226)*z0*z2 + 28.165*z0*z3
            + 0.172*z0*u0 + (-0.655)*z0*u1 + 15.116*z1*z1 + 6.602*z1*z2 + 2.646*z1*u0
            + (-10.407)*z1*u1 + 10.848*z2*z3 + (-11.641)*z3*z3 + 2.264*z3*u0 + (-10.630)*z3*u1
            + 0.263*u0*u0 + (-0.045)*u0*u1 + 0.208*u1*u1 )

    dx1 = ( -0.206 + 0.631*z0 + (-8.241)*z1 + 4.877*z2 + (-3.988)*z3 + (-0.081)*u0
            + 0.246*z0*z0 + 0.205*z0*z1 + 0.263*z0*z2 + 0.023*z0*u0 + (-0.263)*z0*u1
            + 0.314*z1*z2 + 0.020*z1*u0 + 0.064*z2*u0 + (-0.210)*z2*u1 + (-0.079)*z3*z3
            + (-0.057)*z3*u1 + 0.061*u0*u0 + 0.051*u1*u1 )

    dx2 = ( -0.074 + (-9.440)*z0 + 72.535*z1 + (-19.108)*z2 + 61.682*z3
            + (-0.462)*u0 + (-0.441)*u1 + (-0.848)*z0*z0 + (-0.890)*z0*z2 + 0.013*z0*u0
            + 0.510*z0*u1 + (-1.298)*z1*z1 + (-0.079)*z2*u0 + 0.368*z2*u1 + 1.249*z3*z3
            + (-0.068)*z3*u0 + 0.305*z3*u1 + 0.083*u0*u0 + 0.032*u0*u1 + (-0.075)*u1*u1 )

    dx3 = ( 0.198 + 0.433*z0 + 3.306*z1 + (-3.345)*z2 + 0.138*u0 + (-0.379)*u1
            + (-0.283)*z0*z0 + (-0.250)*z0*z1 + (-0.275)*z0*z2 + (-0.011)*z0*u0 + 0.280*z0*u1
            + (-0.369)*z1*z2 + (-0.121)*z1*z3 + (-0.008)*z1*u0 + (-0.076)*z2*u0 + 0.217*z2*u1
            + 0.078*z3*u1 + (-0.046)*u0*u0 + 0.003*u0*u1 + (-0.053)*u1*u1 )

    return ca.vcat([dx0, dx1, dx2, dx3])

def fz_H2(zx, zu):
    # Standardized states/controls
    z0, z1, z2, z3, z4 = zx[0], zx[1], zx[2], zx[3], zx[4]
    u0 = zu[0]

    dx0 = ( 0.005 + (-0.001)*z0 + 74.093*z2 + (-76.455)*z3 + 2.364*u0 )
    dx1 = ( 0.005 + 2.587*z0 + (-0.001)*z1 + 69.005*z2 + (-71.589)*z3 )
    dx2 = ( 0.004 + 5.212*z0 + (-5.209)*z1 + (-0.001)*z2 )
    dx3 = ( 0.005 + 2.570*z0 + 69.084*z2 + (-71.652)*z3 )
    dx4 = ( 0.005 + 2.365*z0 + 74.093*z2 + (-76.455)*z3 + (-0.001)*z4 )

    return ca.vcat([dx0, dx1, dx2, dx3, dx4])

# ---------------------------------------------------------
# Helper functions (integration, scaling, smooth max)
# ---------------------------------------------------------

def rk4(f, z, v, h):
    k1 = f(z, v)
    k2 = f(z + 0.5*h*k1, v)
    k3 = f(z + 0.5*h*k2, v)
    k4 = f(z + h*k3,     v)
    return z + (h/6.0)*(k1 + 2*k2 + 2*k3 + k4)

def rk4_m(f, z, v, dt, m=4):
    h = dt/m
    for _ in range(m):
        z = rk4(f, z, v, h)
    return z

def softplus(x, beta=20.0):
    # smooth max(0,x). Larger beta -> sharper hinge; keep finite to avoid overflow.
    return (1.0/beta) * ca.log(1 + ca.exp(beta*x))

# ==== SAFE NONNEGATIVITY & MATH (stable, smooth) =========================
# If you still want a ReLU name, alias it to the smooth version:
def relu(x):  # smooth, IPOPT-friendly
    return softplus_stable(x, beta=20.0)

def safe_div(numer, denom, eps=1e-9):
    return numer / (denom + eps)

def safe_log(x, eps=1e-12):
    return ca.log(ca.fmax(x, eps))

def safe_sqrt(x, eps=1e-12):
    return ca.sqrt(ca.fmax(x, eps))

def clip_mx(x, lo, hi):
    return ca.fmax(ca.fmin(x, hi), lo)

# ---- ONE place to set this ----
BETA_SP = 20.0   # same value in constraints, warm start, and reporting

def softplus_stable(a, beta=BETA_SP):
    a = ca.fmax(ca.fmin(a, 40.0/beta), -40.0/beta)  # same clipping as before
    return (1.0/beta) * ca.log1p(ca.exp(beta*a))

def softplus_np(a, beta=BETA_SP):
    a = np.clip(a, -40.0/beta, 40.0/beta)
    return (1.0/beta) * np.log1p(np.exp(beta*a))

# Warm-start helper MUST match constraints exactly:
def _sp_np(a, beta=BETA_SP):
    a = np.clip(a, -40.0/beta, 40.0/beta)
    return (1.0/beta) * np.log1p(np.exp(beta*a))

# --- keep this for STANDARDIZED quantities only (small O(1) arguments) ---
def softplus_std(a, beta=BETA_SP):
    a = ca.fmax(ca.fmin(a, 40.0/beta), -40.0/beta)   # OK for standardized numbers
    return (1.0/beta) * ca.log1p(ca.exp(beta*a))

# numpy versions (for warm-start):
def softplus_std_np(a, beta=BETA_SP):
    a = np.clip(a, -40.0/beta, 40.0/beta)
    return (1.0/beta) * np.log1p(np.exp(beta*a))

# ---- Smoothness / stability knobs ---------------------------------
BETA_SP1  = 10.0      # softplus/softclip sharpness (lower = gentler)
SP_CLIP  = 40.0      # guards exp(beta*a) via clipping of 'a' region

def softplus_phys(a, beta=BETA_SP1):
    # Stable, scale-correct softplus(a) with elementwise operations in CasADi (MX):
    # softplus(a) = max(βa,0)/β + log1p(exp(-|βa|))/β
    z  = beta * a
    t  = ca.exp(-ca.fabs(z))                 # exp(-|βa|)
    return (ca.fmax(z, 0) + ca.log1p(t)) / beta

def softclip_mx(a, lo, hi, beta=BETA_SP1):
    # smooth clamp: returns ~min(hi, max(lo, a)) with smooth tansitions
    # uses the SAME stable softplus() above for well-behaved derivatives
    return (lo
            + softplus_phys(a - lo, beta)
            - softplus_phys(a - hi, beta))

def softplus_phys_np(a, beta=BETA_SP1):
    z = beta * a
    out = np.empty_like(a, dtype=float)
    pos = z > 0
    out[pos]  = a[pos] + (1.0/beta) * np.log1p(np.exp(-z[pos]))
    out[~pos] = (1.0/beta) * np.log1p(np.exp(z[~pos]))
    return out

def softclip_np(a, lo, hi, beta=BETA_SP1):
    # lo + softplus(a-lo) - softplus(a-hi)
    return lo + softplus_phys_np(a - lo, beta) - softplus_phys_np(a - hi, beta)
# =========================================================================

def z_hist(Zmat, k, delay):
    idx = k - delay
    if idx < 0:
        return None
    return Zmat[:, idx]

# --- elementwise affine map via diagonal matmul (works on all CasADi versions) ---
def affine_from_z(mu_vec_dm, sig_vec_dm, z_vec):
    # returns mu + diag(sig) @ z   (element-wise: mu + sig * z)
    return mu_vec_dm + ca.mtimes(ca.diag(sig_vec_dm), z_vec)

def affine_from_v(mu_vec_dm, sig_vec_dm, v_vec):
    # returns mu + diag(sig) @ v   (element-wise: mu + sig * v)
    return mu_vec_dm + ca.mtimes(ca.diag(sig_vec_dm), v_vec)

In [3]:
# ---------------------------------------------------------
# BUILD NLP (CasADi)  -- (STANDARDIZED decision variables)
# ---------------------------------------------------------

# Create MX variables in STANDARDIZED space
Z_AA   = ca.MX.sym("Z_AA",   nx_AA,   N+1)
Z_PAP  = ca.MX.sym("Z_PAP",  nx_PAP,  N+1)
Z_APAP = ca.MX.sym("Z_APAP", nx_APAP, N+1)
Z_H2   = ca.MX.sym("Z_H2",   nx_H2,   N+1)

V_AA   = ca.MX.sym("V_AA",   nu_AA,   N)
V_PAP  = ca.MX.sym("V_PAP",  nu_PAP,  N)
V_APAP = ca.MX.sym("V_APAP", nu_APAP, N)
V_H2   = ca.MX.sym("V_H2",   nu_H2,   N)

# Buffers (SCALED to [0,1])
B1s = ca.MX.sym("B_AA_APAP_s",   N+1)
B2s = ca.MX.sym("B_PAP_APAP_s",  N+1)
B3s = ca.MX.sym("B_APAP_H2_s",   N+1)

# Slacks for demand (soft constraints)
s_dem_apap = ca.MX.sym("s_dem_apap")
s_dem_h2   = ca.MX.sym("s_dem_h2")

# ==== STABILIZE STANDARDIZATION (no asserts; just fix) ===================
_sigma_floor = 1e-3  # safe default; tune if needed
sigma_x_AA   = np.maximum(sigma_x_AA,   _sigma_floor)
sigma_u_AA   = np.maximum(sigma_u_AA,   _sigma_floor)
sigma_x_PAP  = np.maximum(sigma_x_PAP,  _sigma_floor)
sigma_u_PAP  = np.maximum(sigma_u_PAP,  _sigma_floor)
sigma_x_APAP = np.maximum(sigma_x_APAP, _sigma_floor)
sigma_u_APAP = np.maximum(sigma_u_APAP, _sigma_floor)
sigma_x_H2   = np.maximum(sigma_x_H2,   _sigma_floor)
sigma_u_H2   = np.maximum(sigma_u_H2,   _sigma_floor)

# Physical mapping helpers
mu_x_AA_dm,   sig_x_AA_dm   = ca.DM(mu_x_AA),   ca.DM(sigma_x_AA)
mu_u_AA_dm,   sig_u_AA_dm   = ca.DM(mu_u_AA),   ca.DM(sigma_u_AA)
mu_x_PAP_dm,  sig_x_PAP_dm  = ca.DM(mu_x_PAP),  ca.DM(sigma_x_PAP)
mu_u_PAP_dm,  sig_u_PAP_dm  = ca.DM(mu_u_PAP),  ca.DM(sigma_u_PAP)
mu_x_APAP_dm, sig_x_APAP_dm = ca.DM(mu_x_APAP), ca.DM(sigma_x_APAP)
mu_u_APAP_dm, sig_u_APAP_dm = ca.DM(mu_u_APAP), ca.DM(sigma_u_APAP)
mu_x_H2_dm,   sig_x_H2_dm   = ca.DM(mu_x_H2),   ca.DM(sigma_x_H2)
mu_u_H2_dm,   sig_u_H2_dm   = ca.DM(mu_u_H2),   ca.DM(sigma_u_H2)
# =========================================================================

def xA(k):    return affine_from_z(mu_x_AA_dm,   sig_x_AA_dm,   Z_AA[:,k])
def uA(k):    return affine_from_v(mu_u_AA_dm,   sig_u_AA_dm,   V_AA[:,k])
def xP(k):    return affine_from_z(mu_x_PAP_dm,  sig_x_PAP_dm,  Z_PAP[:,k])
def uP(k):    return affine_from_v(mu_u_PAP_dm,  sig_u_PAP_dm,  V_PAP[:,k])
def xAP(k):   return affine_from_z(mu_x_APAP_dm, sig_x_APAP_dm, Z_APAP[:,k])
def uAP(k):   return affine_from_v(mu_u_APAP_dm, sig_u_APAP_dm, V_APAP[:,k])
def xH2(k):   return affine_from_z(mu_x_H2_dm,   sig_x_H2_dm,   Z_H2[:,k])
def uH2(k):   return affine_from_v(mu_u_H2_dm,   sig_u_H2_dm,   V_H2[:,k])

# Constraint lists & objective
g   = []
g_lb= []
g_ub= []
J   = 0.0

# --- Initial conditions (STANDARDIZED) ---
Z0_AA   = (ca.DM(x0_AA)   - mu_x_AA_dm)/sig_x_AA_dm
Z0_PAP  = (ca.DM(x0_PAP)  - mu_x_PAP_dm)/sig_x_PAP_dm
Z0_APAP = (ca.DM(x0_APAP) - mu_x_APAP_dm)/sig_x_APAP_dm
Z0_H2   = (ca.DM(x0_H2)   - mu_x_H2_dm)/sig_x_H2_dm

g += [ Z_AA[:,0]   - Z0_AA,
       Z_PAP[:,0]  - Z0_PAP,
       Z_APAP[:,0] - Z0_APAP,
       Z_H2[:,0]   - Z0_H2 ]
g_lb += [0.0]*nx_AA + [0.0]*nx_PAP + [0.0]*nx_APAP + [0.0]*nx_H2
g_ub += [0.0]*nx_AA + [0.0]*nx_PAP + [0.0]*nx_APAP + [0.0]*nx_H2

# --- Initial buffers (scaled) ---
g += [B1s[0] - (B0_AA_APAP  / Bmax_AA_APAP),
      B2s[0] - (B0_PAP_APAP / Bmax_PAP_APAP),
      B3s[0] - (B0_APAP_H2  / Bmax_APAP_H2)]
g_lb += [0.0, 0.0, 0.0]
g_ub += [0.0, 0.0, 0.0]

# --- Dynamics, buffers, objective, demand ---
RK_SUBSTEPS = 10
cum_APAP = 0.0
cum_H2   = 0.0

# ==== DYNAMICS WRAPPERS: make ODE evals domain-safe ======================
Z_CLIP  = 20.0   # was 8.0
V_CLIP  = 10.0   # was 5.0
DZ_CLIP = 1e5    # was 100.0

def fz_AA_safe(z, v):
    zc = softclip_mx(z, -Z_CLIP, Z_CLIP, beta=BETA_SP1)
    vc = softclip_mx(v, -V_CLIP, V_CLIP, beta=BETA_SP1)
    dz = fz_AA(zc, vc)
    return softclip_mx(dz, -DZ_CLIP, DZ_CLIP, beta=BETA_SP1)

def fz_PAP_safe(z, v):
    zc = softclip_mx(z, -Z_CLIP, Z_CLIP, beta=BETA_SP1)
    vc = softclip_mx(v, -V_CLIP, V_CLIP, beta=BETA_SP1)
    dz = fz_PAP(zc, vc)
    return softclip_mx(dz, -DZ_CLIP, DZ_CLIP, beta=BETA_SP1)

def fz_APAP_safe(z, v):
    zc = softclip_mx(z, -Z_CLIP, Z_CLIP, beta=BETA_SP1)
    vc = softclip_mx(v, -V_CLIP, V_CLIP, beta=BETA_SP1)
    dz = fz_APAP(zc, vc)
    return softclip_mx(dz, -DZ_CLIP, DZ_CLIP, beta=BETA_SP1)

def fz_H2_safe(z, v):
    zc = softclip_mx(z, -Z_CLIP, Z_CLIP, beta=BETA_SP1)
    vc = softclip_mx(v, -V_CLIP, V_CLIP, beta=BETA_SP1)
    dz = fz_H2(zc, vc)
    return softclip_mx(dz, -DZ_CLIP, DZ_CLIP, beta=BETA_SP1)
# =========================================================================

for k in range(N):
    # Plant dynamics in STANDARDIZED space (z' = fz(z,v)), integrated by RK4 with substeps
    zA_next   = rk4_m(fz_AA_safe,   Z_AA[:,k],   V_AA[:,k],   Delta_t, m=RK_SUBSTEPS)
    zP_next   = rk4_m(fz_PAP_safe,  Z_PAP[:,k],  V_PAP[:,k],  Delta_t, m=RK_SUBSTEPS)
    zAP_next  = rk4_m(fz_APAP_safe, Z_APAP[:,k], V_APAP[:,k], Delta_t, m=RK_SUBSTEPS)
    zH2_next  = rk4_m(fz_H2_safe,   Z_H2[:,k],   V_H2[:,k],   Delta_t, m=RK_SUBSTEPS)

    g += [ Z_AA[:,k+1]   - zA_next,
           Z_PAP[:,k+1]  - zP_next,
           Z_APAP[:,k+1] - zAP_next,
           Z_H2[:,k+1]   - zH2_next ]
    g_lb += [0.0]*nx_AA + [0.0]*nx_PAP + [0.0]*nx_APAP + [0.0]*nx_H2
    g_ub += [0.0]*nx_AA + [0.0]*nx_PAP + [0.0]*nx_APAP + [0.0]*nx_H2

    # Arrivals with delays (in PHYSICAL kg/h), using smooth nonnegativity
    zAA_hist  = z_hist(Z_AA,   k, d_AA_to_APAP)
    zAP_hist  = z_hist(Z_APAP, k, d_APAP_to_H2)

    # AA→APAP arrival
    if zAA_hist is None:
        arr_AA_APAP = 0.0
    else:
        xAA_hist = affine_from_z(mu_x_AA_dm, sig_x_AA_dm, zAA_hist)
        arr_AA_APAP = softplus_phys( xAA_hist[IDX_AA_TO_APAP] )

    # PAP→APAP arrival (no delay)
    arr_PAP_APAP = softplus_phys( xP(k)[IDX_PAP_TO_APAP] )

    # APAP→H2 arrival (with delay)
    if zAP_hist is None:
        arr_APAP_H2 = 0.0
    else:
        xAP_hist = affine_from_z(mu_x_APAP_dm, sig_x_APAP_dm, zAP_hist)
        arr_APAP_H2 = softplus_phys( xAP_hist[IDX_APAP_WASTE_TO_H2] )
        
    uAP_dev = ca.mtimes(ca.diag(sig_u_APAP_dm), V_APAP[:,k])   # = σ_u_APAP ⊙ v_APAP
    uH2_dev = ca.mtimes(ca.diag(sig_u_H2_dm),   V_H2[:,k])     # = σ_u_H2   ⊙ v_H2

    # Withdrawals from buffers (map to APAP/H2 controls; smooth nonnegativity)
    withdraw_AA_APAP   = softplus_phys( uAP(k)[U_APAP_WITHDRAW_IDX_AA] )
    withdraw_PAP_APAP  = softplus_phys( uAP(k)[U_APAP_WITHDRAW_IDX_PAP] )
    withdraw_APAP_H2   = softplus_phys( uH2(k)[U_H2_WITHDRAW_IDX_APAPWASTE] )

    # Buffer balances in PHYSICAL units, constrained via SCALED variables
    rhs1 = B1s[k] + (Delta_t / Bmax_AA_APAP)   * (arr_AA_APAP   - withdraw_AA_APAP)
    rhs2 = B2s[k] + (Delta_t / Bmax_PAP_APAP)  * (arr_PAP_APAP  - withdraw_PAP_APAP)
    rhs3 = B3s[k] + (Delta_t / Bmax_APAP_H2)   * (arr_APAP_H2   - withdraw_APAP_H2)

    # Pure mass-balance equality; keep 0..1 via variable bounds (already set below)
    g += [ B1s[k+1] - rhs1,
           B2s[k+1] - rhs2,
           B3s[k+1] - rhs3 ]

    g_lb += [0.0, 0.0, 0.0]
    g_ub += [0.0, 0.0, 0.0]

    # Objective: integrate waste rates (PHYSICAL kg/h)
    waste_AA   = ca.sum1(ca.vcat([softplus_phys(xA(k)[i])   for i in WASTE_IDX_AA]))
    waste_PAP  = ca.sum1(ca.vcat([softplus_phys(xP(k)[i])   for i in WASTE_IDX_PAP]))
    waste_APAP = ca.sum1(ca.vcat([softplus_phys(xAP(k)[i])  for i in WASTE_IDX_APAP]))
    J = J + Delta_t * (waste_AA + waste_PAP + waste_APAP)

    # Demand accumulation (PHYSICAL kg/h)
    y_apap = softplus_phys( xAP(k)[IDX_APAP_PROD] )
    y_h2   = softplus_phys( xH2(k)[IDX_H2_PROD] )
    cum_APAP = cum_APAP + Delta_t * y_apap
    cum_H2   = cum_H2   + Delta_t * y_h2

# Terminal demand: SOFT constraints with slacks (≥ demand)
D_SCALE_APAP = max(1.0, D_APAP)
D_SCALE_H2   = max(1.0, D_H2)

# Remember indices for diagnostics
idx_dem_apap = len(g)
g += [ (cum_APAP + s_dem_apap - D_APAP) / D_SCALE_APAP ]
g_lb += [0.0]
g_ub += [1e19]

idx_dem_h2 = len(g)
g += [ (cum_H2   + s_dem_h2   - D_H2)   / D_SCALE_H2   ]
g_lb += [0.0]
g_ub += [1e19]

# ---- Demand overshoot penalty (makes controls move to just-meet demand) ----
W_OVERSHOOT = 1e4  # tune 1e3–1e5
over_APAP = softplus_phys((cum_APAP - D_APAP) / D_SCALE_APAP)  # >0 iff cum_APAP > D_APAP
over_H2   = softplus_phys((cum_H2   - D_H2)   / D_SCALE_H2)
J = J + W_OVERSHOOT * (over_APAP**2 + over_H2**2)

# Slack penalties
W_SLACK = 1e5
J = J + W_SLACK * (s_dem_apap + s_dem_h2)

# ========== FEASIBLE WARM START (rollout with safe integrators) ==========
# Build discrete-time flow maps once (standardized space)
zA_sym = ca.MX.sym('zA', nx_AA);   vA_sym = ca.MX.sym('vA', nu_AA)
zP_sym = ca.MX.sym('zP', nx_PAP);  vP_sym = ca.MX.sym('vP', nu_PAP)
zX_sym = ca.MX.sym('zX', nx_APAP); vX_sym = ca.MX.sym('vX', nu_APAP)
zH_sym = ca.MX.sym('zH', nx_H2);   vH_sym = ca.MX.sym('vH', nu_H2)

Phi_AA   = ca.Function('Phi_AA',   [zA_sym, vA_sym], [rk4_m(fz_AA_safe,   zA_sym, vA_sym, Delta_t, m=RK_SUBSTEPS)])
Phi_PAP  = ca.Function('Phi_PAP',  [zP_sym, vP_sym], [rk4_m(fz_PAP_safe,  zP_sym, vP_sym, Delta_t, m=RK_SUBSTEPS)])
Phi_APAP = ca.Function('Phi_APAP', [zX_sym, vX_sym], [rk4_m(fz_APAP_safe, zX_sym, vX_sym, Delta_t, m=RK_SUBSTEPS)])
Phi_H2   = ca.Function('Phi_H2',   [zH_sym, vH_sym], [rk4_m(fz_H2_safe,   zH_sym, vH_sym, Delta_t, m=RK_SUBSTEPS)])

# Zero-controls rollout for initialization
V_AA_init   = np.zeros((nu_AA,   N))
V_PAP_init  = np.zeros((nu_PAP,  N))
V_APAP_init = np.zeros((nu_APAP, N))
V_H2_init   = np.zeros((nu_H2,   N))

# Start from your standardized initial conditions
Z_AA_init   = np.zeros((nx_AA,   N+1)); Z_AA_init[:,0]   = np.array(Z0_AA).ravel()
Z_PAP_init  = np.zeros((nx_PAP,  N+1)); Z_PAP_init[:,0]  = np.array(Z0_PAP).ravel()
Z_APAP_init = np.zeros((nx_APAP, N+1)); Z_APAP_init[:,0] = np.array(Z0_APAP).ravel()
Z_H2_init   = np.zeros((nx_H2,   N+1)); Z_H2_init[:,0]   = np.array(Z0_H2).ravel()

# Rollout
for k in range(N):
    Z_AA_init[:,k+1]   = np.array(Phi_AA(  Z_AA_init[:,k],   V_AA_init[:,k])).ravel()
    Z_PAP_init[:,k+1]  = np.array(Phi_PAP( Z_PAP_init[:,k],  V_PAP_init[:,k])).ravel()
    Z_APAP_init[:,k+1] = np.array(Phi_APAP(Z_APAP_init[:,k], V_APAP_init[:,k])).ravel()
    Z_H2_init[:,k+1]   = np.array(Phi_H2(  Z_H2_init[:,k],   V_H2_init[:,k])).ravel()

B1s_init = np.zeros(N+1); B1s_init[0] = B0_AA_APAP  / Bmax_AA_APAP
B2s_init = np.zeros(N+1); B2s_init[0] = B0_PAP_APAP / Bmax_PAP_APAP
B3s_init = np.zeros(N+1); B3s_init[0] = B0_APAP_H2  / Bmax_APAP_H2

for k in range(N):
    # arrivals at step k (PHYSICAL), matching constraint formulas
    if (k - d_AA_to_APAP) < 0:
        arr_AA_APAP = 0.0
    else:
        xAA_hist = mu_x_AA + sigma_x_AA * Z_AA_init[:, k - d_AA_to_APAP]
        arr_AA_APAP = softplus_phys_np(xAA_hist[IDX_AA_TO_APAP])

    xP_k = mu_x_PAP + sigma_x_PAP * Z_PAP_init[:, k]
    arr_PAP_APAP = softplus_phys_np(xP_k[IDX_PAP_TO_APAP])

    if (k - d_APAP_to_H2) < 0:
        arr_APAP_H2 = 0.0
    else:
        xAP_hist = mu_x_APAP + sigma_x_APAP * Z_APAP_init[:, k - d_APAP_to_H2]
        arr_APAP_H2 = softplus_phys_np(xAP_hist[IDX_APAP_WASTE_TO_H2])

    # withdrawals (PHYSICAL) using zero control warm start
    uAP_dev_k = sigma_u_APAP * V_APAP_init[:, k]   # elementwise
    uH2_dev_k = sigma_u_H2  * V_H2_init[:,  k]
    withdraw_AA_APAP  = softplus_phys_np(uAP_dev_k[U_APAP_WITHDRAW_IDX_AA])
    withdraw_PAP_APAP = softplus_phys_np(uAP_dev_k[U_APAP_WITHDRAW_IDX_PAP])
    withdraw_APAP_H2  = softplus_phys_np(uH2_dev_k[U_H2_WITHDRAW_IDX_APAPWASTE])

    rhs1 = B1s_init[k] + (Delta_t / Bmax_AA_APAP)  * (arr_AA_APAP  - withdraw_AA_APAP)
    rhs2 = B2s_init[k] + (Delta_t / Bmax_PAP_APAP) * (arr_PAP_APAP - withdraw_PAP_APAP)
    rhs3 = B3s_init[k] + (Delta_t / Bmax_APAP_H2)  * (arr_APAP_H2  - withdraw_APAP_H2)
    
    B1s_init[k+1] = rhs1
    B2s_init[k+1] = rhs2
    B3s_init[k+1] = rhs3


# ---- Warm-start the demand slacks to satisfy the terminal inequalities ----
cum_APAP_init = 0.0
cum_H2_init   = 0.0
for k in range(N):
    xAP_k = mu_x_APAP + sigma_x_APAP * Z_APAP_init[:, k]
    xH2_k = mu_x_H2   + sigma_x_H2   * Z_H2_init[:,   k]
    y_apap_k = softplus_phys_np(xAP_k[IDX_APAP_PROD])
    y_h2_k   = softplus_phys_np(xH2_k[IDX_H2_PROD])
    cum_APAP_init += Delta_t * y_apap_k
    cum_H2_init   += Delta_t * y_h2_k

s_dem_apap_init = float(max(0.0, D_APAP - cum_APAP_init))
s_dem_h2_init   = float(max(0.0, D_H2   - cum_H2_init))

# ---------------------------------------------------------
# Variable packing, bounds, trust regions, regularization
# ---------------------------------------------------------

w   = []
lbw = []
ubw = []
w0  = []

def append_var(vsym, lb=None, ub=None, init=None):
    v_flat = ca.vec(vsym)
    w.append(v_flat)
    n = int(v_flat.shape[0])
    if lb is None:   lb = [-1e19]*n
    if ub is None:   ub = [ 1e19]*n
    if init is None: init = [0.0]*n
    lbw.extend(lb); ubw.extend(ub); w0.extend(init)

# Trust regions on standardized states/controls
DV_MAX = 20.0
V_MAX  = 20.0 
Z_MAX   = 10000.0

# Add variables with box bounds and warm-start inits (FORTRAN order)
append_var(Z_AA,   lb=[-Z_MAX]*(nx_AA*(N+1)),   ub=[ Z_MAX]*(nx_AA*(N+1)),
           init=Z_AA_init.flatten(order="F").tolist())
append_var(Z_PAP,  lb=[-Z_MAX]*(nx_PAP*(N+1)),  ub=[ Z_MAX]*(nx_PAP*(N+1)),
           init=Z_PAP_init.flatten(order="F").tolist())
append_var(Z_APAP, lb=[-Z_MAX]*(nx_APAP*(N+1)), ub=[ Z_MAX]*(nx_APAP*(N+1)),
           init=Z_APAP_init.flatten(order="F").tolist())
append_var(Z_H2,   lb=[-Z_MAX]*(nx_H2*(N+1)),   ub=[ Z_MAX]*(nx_H2*(N+1)),
           init=Z_H2_init.flatten(order="F").tolist())

append_var(V_AA,   lb=[-V_MAX]*(nu_AA*N),   ub=[ V_MAX]*(nu_AA*N),
           init=V_AA_init.flatten(order="F").tolist())
append_var(V_PAP,  lb=[-V_MAX]*(nu_PAP*N),  ub=[ V_MAX]*(nu_PAP*N),
           init=V_PAP_init.flatten(order="F").tolist())
append_var(V_APAP, lb=[-V_MAX]*(nu_APAP*N), ub=[ V_MAX]*(nu_APAP*N),
           init=V_APAP_init.flatten(order="F").tolist())
append_var(V_H2,   lb=[-V_MAX]*(nu_H2*N),   ub=[ V_MAX]*(nu_H2*N),
           init=V_H2_init.flatten(order="F").tolist())

append_var(B1s, lb=[0.0]*(N+1), ub=[1.0]*(N+1), init=B1s_init.tolist())
append_var(B2s, lb=[0.0]*(N+1), ub=[1.0]*(N+1), init=B2s_init.tolist())
append_var(B3s, lb=[0.0]*(N+1), ub=[1.0]*(N+1), init=B3s_init.tolist())

# Demand slacks (>=0)
append_var(s_dem_apap, lb=[0.0], ub=[1e19], init=[s_dem_apap_init])
append_var(s_dem_h2,   lb=[0.0], ub=[1e19], init=[s_dem_h2_init])

# Δv (move limits) as constraints
for k in range(N-1):
    g += [ V_AA[:,k+1]-V_AA[:,k],
           V_PAP[:,k+1]-V_PAP[:,k],
           V_APAP[:,k+1]-V_APAP[:,k],
           V_H2[:,k+1]-V_H2[:,k] ]
    g_lb += [-DV_MAX]*nu_AA  + [-DV_MAX]*nu_PAP + [-DV_MAX]*nu_APAP + [-DV_MAX]*nu_H2
    g_ub += [ DV_MAX]*nu_AA  + [ DV_MAX]*nu_PAP + [ DV_MAX]*nu_APAP + [ DV_MAX]*nu_H2

# Regularization on controls and smoothness
LAMBDA_U  = 1e-4
LAMBDA_DU = 1e-3
J = J + LAMBDA_U * (ca.sumsqr(V_AA) + ca.sumsqr(V_PAP) + ca.sumsqr(V_APAP) + ca.sumsqr(V_H2))
for k in range(N-1):
    J = J + LAMBDA_DU * ( ca.sumsqr(V_AA[:,k+1]-V_AA[:,k])
                        + ca.sumsqr(V_PAP[:,k+1]-V_PAP[:,k])
                        + ca.sumsqr(V_APAP[:,k+1]-V_APAP[:,k])
                        + ca.sumsqr(V_H2[:,k+1]-V_H2[:,k]) )

# ---------------------------------------------------------
# Solve NLP (ROBUST SETUP)
# ---------------------------------------------------------
g = ca.vertcat(*g)
w = ca.vertcat(*w)

nlp = {"x": w, "f": J, "g": g}

opts = {
    "error_on_fail": False,
    "expand": False,
    "print_time": False,
    "ipopt": {
        "max_iter": 1000,
        "hessian_approximation": "limited-memory",   # was "exact"
        "limited_memory_max_history": 20,
        "nlp_scaling_method": "gradient-based",
        "mu_strategy": "adaptive",
        "mu_init": 1e-3,                              # was 1e-1
        "tol": 1e-4,
        "constr_viol_tol": 1e-4,
        "compl_inf_tol": 1e-4,
        "bound_push": 1e-2,                           # was 1e-3
        "bound_frac": 5e-2,                           # was 1e-2
        "bound_relax_factor": 1e-5,
        "honor_original_bounds": "yes",
        "linear_solver": "mumps",
        "check_derivatives_for_naninf": "yes",
        "print_level": 5,
        "acceptable_tol": 1e-3,
        "acceptable_iter": 5
    }
}


# --- Remove Ipopt options not supported by this build ---
opts["ipopt"].pop("acceptable_strategy", None)
opts["ipopt"].pop("regularization_type", None)

solver = ca.nlpsol("solver", "ipopt", nlp, opts)

# --- Solve (NO exceptions on "Max_Iter_Exceeded", "Restoration_Failed", etc.)
sol = solver(x0=w0, lbx=lbw, ubx=ubw,
             lbg=[float(v) for v in g_lb],
             ubg=[float(v) for v in g_ub])

# Diagnostics: check both terminal demand constraints at warm start and at solution
g_fun = ca.Function('g_fun', [w], [g])
g_all0 = np.array(g_fun(ca.vertcat(*w0))).ravel()
g_all  = np.array(g_fun(ca.vertcat(*np.array(sol["x"]).ravel()))).ravel()

gd0_apap = float(g_all0[idx_dem_apap])
gd_apap  = float(g_all[idx_dem_apap])
gd0_h2   = float(g_all0[idx_dem_h2])
gd_h2    = float(g_all[idx_dem_h2])

print(f"[diag] APAP demand residual (scaled) warm-start: {gd0_apap:.6f}  (>=0 feasible)")
print(f"[diag] APAP demand residual (scaled) solution   : {gd_apap:.6f}  (>=0 feasible)")
print(f"[diag] H2   demand residual (scaled) warm-start: {gd0_h2:.6f}   (>=0 feasible)")
print(f"[diag] H2   demand residual (scaled) solution   : {gd_h2:.6f}   (>=0 feasible)")
print(f"[diag] D_APAP = {D_APAP}, D_H2 = {D_H2}, D_SCALE_APAP = {D_SCALE_APAP}, D_SCALE_H2 = {D_SCALE_H2}")

w_opt = np.array(sol["x"]).ravel()

# ---------------------------------------------------------
# Unpack solution (STANDARDIZED), then convert to PHYSICAL & SAVE
# ---------------------------------------------------------
def unstack(w_opt, nx_AA, nu_AA, nx_PAP, nu_PAP, nx_APAP, nu_APAP, nx_H2, nu_H2, N):
    ptr = 0
    def take(n):
        nonlocal ptr
        out = w_opt[ptr:ptr+n]
        ptr += n
        return out

    Z_AA_opt   = np.reshape(take(nx_AA*(N+1)),   (nx_AA,   N+1), order="F")
    Z_PAP_opt  = np.reshape(take(nx_PAP*(N+1)),  (nx_PAP,  N+1), order="F")
    Z_APAP_opt = np.reshape(take(nx_APAP*(N+1)), (nx_APAP, N+1), order="F")
    Z_H2_opt   = np.reshape(take(nx_H2*(N+1)),   (nx_H2,   N+1), order="F")

    V_AA_opt   = np.reshape(take(nu_AA*N),   (nu_AA,   N), order="F")
    V_PAP_opt  = np.reshape(take(nu_PAP*N),  (nu_PAP,  N), order="F")
    V_APAP_opt = np.reshape(take(nu_APAP*N), (nu_APAP, N), order="F")
    V_H2_opt   = np.reshape(take(nu_H2*N),   (nu_H2,   N), order="F")

    B1s_opt    = take(N+1)
    B2s_opt    = take(N+1)
    B3s_opt    = take(N+1)

    s_dem_apap_opt = take(1)[0]
    s_dem_h2_opt   = take(1)[0]

    return (Z_AA_opt, Z_PAP_opt, Z_APAP_opt, Z_H2_opt,
            V_AA_opt, V_PAP_opt, V_APAP_opt, V_H2_opt,
            np.array(B1s_opt), np.array(B2s_opt), np.array(B3s_opt),
            float(s_dem_apap_opt), float(s_dem_h2_opt))

(
    Z_AA_opt, Z_PAP_opt, Z_APAP_opt, Z_H2_opt,
    V_AA_opt, V_PAP_opt, V_APAP_opt, V_H2_opt,
    B1s_opt, B2s_opt, B3s_opt,
    s_dem_apap_opt, s_dem_h2_opt
) = unstack(w_opt, nx_AA, nu_AA, nx_PAP, nu_PAP, nx_APAP, nu_APAP, nx_H2, nu_H2, N)

# Convert STANDARDIZED -> PHYSICAL
X_AA_phys    = (mu_x_AA[:,   None] + sigma_x_AA[:,   None] * Z_AA_opt)     # (nx_AA,   N+1)
X_PAP_phys   = (mu_x_PAP[:,  None] + sigma_x_PAP[:,  None] * Z_PAP_opt)    # (nx_PAP,  N+1)
X_APAP_phys  = (mu_x_APAP[:, None] + sigma_x_APAP[:, None] * Z_APAP_opt)   # (nx_APAP, N+1)
X_H2_phys    = (mu_x_H2[:,   None] + sigma_x_H2[:,   None] * Z_H2_opt)     # (nx_H2,   N+1)

U_AA_phys    = (mu_u_AA[:,   None] + sigma_u_AA[:,   None] * V_AA_opt)     # (nu_AA,   N)
U_PAP_phys   = (mu_u_PAP[:,  None] + sigma_u_PAP[:,  None] * V_PAP_opt)    # (nu_PAP,  N)
U_APAP_phys  = (mu_u_APAP[:, None] + sigma_u_APAP[:, None] * V_APAP_opt)   # (nu_APAP, N)
U_H2_phys    = (mu_u_H2[:,   None] + sigma_u_H2[:,   None] * V_H2_opt)     # (nu_H2,   N)

B_AA_APAP_phys  = B1s_opt * Bmax_AA_APAP
B_PAP_APAP_phys = B2s_opt * Bmax_PAP_APAP
B_APAP_H2_phys  = B3s_opt * Bmax_APAP_H2

# Save results
t  = np.arange(N+1)*Delta_t
tu = np.arange(N)*Delta_t

df_x_AA  = pd.DataFrame(X_AA_phys.T,    columns=[f"xAA_{i}"    for i in range(nx_AA)])
df_x_PAP = pd.DataFrame(X_PAP_phys.T,   columns=[f"xPAP_{i}"   for i in range(nx_PAP)])
df_x_APA = pd.DataFrame(X_APAP_phys.T,  columns=[f"xAPAP_{i}"  for i in range(nx_APAP)])
df_x_H2  = pd.DataFrame(X_H2_phys.T,    columns=[f"xH2_{i}"    for i in range(nx_H2)])

for df in (df_x_AA, df_x_PAP, df_x_APA, df_x_H2):
    df.insert(0, "t_h", t)

df_u_AA  = pd.DataFrame(U_AA_phys.T,    columns=[f"uAA_{j}"    for j in range(nu_AA)])
df_u_PAP = pd.DataFrame(U_PAP_phys.T,   columns=[f"uPAP_{j}"   for j in range(nu_PAP)])
df_u_APA = pd.DataFrame(U_APAP_phys.T,  columns=[f"uAPAP_{j}"  for j in range(nu_APAP)])
df_u_H2  = pd.DataFrame(U_H2_phys.T,    columns=[f"uH2_{j}"    for j in range(nu_H2)])

for dfu in (df_u_AA, df_u_PAP, df_u_APA, df_u_H2):
    dfu.insert(0, "t_h", tu)

df_b = pd.DataFrame({
    "t_h": t,
    "B_AA_APAP":   B_AA_APAP_phys,
    "B_PAP_APAP":  B_PAP_APAP_phys,
    "B_APAP_H2":   B_APAP_H2_phys,
})

df_x_AA.to_csv("opt_states_AA.csv",    index=False)
df_x_PAP.to_csv("opt_states_PAP.csv",  index=False)
df_x_APA.to_csv("opt_states_APAP.csv", index=False)
df_x_H2.to_csv("opt_states_H2.csv",    index=False)

df_u_AA.to_csv("opt_controls_AA.csv",    index=False)
df_u_PAP.to_csv("opt_controls_PAP.csv",  index=False)
df_u_APA.to_csv("opt_controls_APAP.csv", index=False)
df_u_H2.to_csv("opt_controls_H2.csv",    index=False)

df_b.to_csv("opt_buffers.csv", index=False)

print("Solved. Objective J =", float(sol["f"]))
print("Wrote opt_states_*.csv, opt_controls_*.csv, opt_buffers.csv")

# We align per-step quantities with k = 0..N-1, so use states at columns 0..N-1
def softplus_np(a, beta=50.0):
    return (1.0/beta) * np.log1p(np.exp(beta*np.clip(a, -40/beta, 40/beta)))

# Per-plant waste rates [kg/h] at each step (length N) -- H2 not included in waste objective by design
w_AA   = np.sum(softplus_np(X_AA_phys[WASTE_IDX_AA,    0:N]), axis=0)
w_PAP  = np.sum(softplus_np(X_PAP_phys[WASTE_IDX_PAP,   0:N]), axis=0)
w_APAP = np.sum(softplus_np(X_APAP_phys[WASTE_IDX_APAP, 0:N]), axis=0)

w_total     = w_AA + w_PAP + w_APAP
w_step_kg   = w_total * Delta_t
w_cum_kg    = np.cumsum(w_step_kg)

# APAP & H2 production rate [kg/h] and integrals
y_apap_rate = softplus_np(X_APAP_phys[IDX_APAP_PROD, 0:N])
y_h2_rate   = softplus_np(X_H2_phys[IDX_H2_PROD,     0:N])
prod_apap_step_kg = y_apap_rate * Delta_t
prod_h2_step_kg   = y_h2_rate   * Delta_t
prod_apap_cum_kg  = np.cumsum(prod_apap_step_kg)
prod_h2_cum_kg    = np.cumsum(prod_h2_step_kg)

t_steps = np.arange(N) * Delta_t

df_obj = pd.DataFrame({
    "t_h": t_steps,
    "w_AA_kgph":    w_AA,
    "w_PAP_kgph":   w_PAP,
    "w_APAP_kgph":  w_APAP,
    "w_total_kgph": w_total,
    "w_step_kg":    w_step_kg,
    "w_cum_kg":     w_cum_kg,
    "apap_rate_kgph": y_apap_rate,
    "apap_step_kg":   prod_apap_step_kg,
    "apap_cum_kg":    prod_apap_cum_kg,
    "h2_rate_kgph":   y_h2_rate,
    "h2_step_kg":     prod_h2_step_kg,
    "h2_cum_kg":      prod_h2_cum_kg
})
df_obj.to_csv("objective_components_physical.csv", index=False)
print("Wrote objective_components_physical.csv")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:    16087
Number of nonzeros in inequality constraint Jacobian.:     1984
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     3124
                     variables with only lower bounds:        2
                variables with lower and upper bounds:     3122
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2222
Total number of inequality c

  78  5.4036542e+09 3.08e+04 5.94e+15   6.3 1.60e+04    -  1.82e-02 1.48e-03h  1
  79r 5.4036542e+09 3.08e+04 1.00e+03   5.6 0.00e+00    -  0.00e+00 4.65e-07R 13
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80r 5.3987677e+09 3.07e+04 9.85e+07   5.3 1.27e+04    -  9.47e-01 5.19e-03f  1
  81r 5.6801589e+09 9.79e+04 5.35e+08   5.1 2.98e+03    -  9.88e-01 6.49e-01f  1
  82  5.6757142e+09 9.80e+04 8.69e+13   4.9 6.05e+01   9.4 1.00e+00 3.46e-01h  1
  83  5.4774234e+09 9.51e+04 6.55e+19   4.9 1.06e+04   9.8 2.49e-02 2.86e-02h  1
  84  5.4758647e+09 9.51e+04 4.64e+20   4.9 2.73e+03   8.8 3.04e-02 1.11e-03h  1
  85  5.4758263e+09 9.51e+04 2.23e+19   4.9 5.86e+08    -  2.89e-08 2.89e-08s  2
  86r 5.4758263e+09 9.51e+04 9.89e+02   4.9 0.00e+00    -  0.00e+00 0.00e+00R  1
  87r 5.4331159e+09 9.38e+04 7.69e+06  -1.2 1.91e+05    -  1.09e-01 2.10e-02f  1
  88r 5.1928431e+09 8.96e+04 1.68e+07   4.2 8.25e+03    -  5.79e-01 1.22e-01f  1
  89r 5.1267457e+09 9.22e+04

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170r 4.2601889e+09 9.97e+04 9.99e+02   3.4 0.00e+00    -  0.00e+00 0.00e+00R  1
 171r 4.2518544e+09 9.97e+04 1.23e+09  -2.9 3.43e+05    -  3.79e-03 2.92e-03f  1
 172r 4.2184320e+09 9.97e+04 1.03e+09   3.0 6.57e+04    -  2.06e-02 5.32e-03f  1
 173r 4.2043916e+09 9.97e+04 2.41e+07   4.9 2.02e+04    -  7.84e-02 1.36e-02f  1
 174r 4.1821085e+09 9.97e+04 2.15e+08   3.6 7.51e+04    -  3.88e-02 1.58e-02f  1
 175r 4.0649672e+09 9.97e+04 2.70e+08   3.6 4.14e+03    -  2.25e-02 4.25e-02f  1
 176r 4.0580102e+09 9.97e+04 2.34e+08   4.6 2.02e+04    -  2.77e-01 1.48e-02f  1
 177  4.0578508e+09 9.97e+04 2.34e+15   2.6 1.63e+00  12.5 1.00e+00 1.00e+00h  1
 178  4.0578543e+09 9.97e+04 5.37e+17   4.6 2.45e-01  14.2 1.00e+00 1.00e+00f  1
 179  4.0579217e+09 9.97e+04 3.62e+15   6.4 7.15e-01  13.7 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180  4.0578812e+09 9.97e+04

 262  4.9108130e+09 6.99e+04 5.09e+11   2.6 6.30e+03    -  2.57e-03 8.55e-06h  8
 263  4.9106972e+09 6.99e+04 5.09e+11   2.6 7.42e+03    -  3.01e-03 5.01e-05f  9
 264  4.9089730e+09 6.98e+04 5.18e+11   2.6 7.34e+03    -  8.40e-03 7.85e-04f  5
 265  4.9069892e+09 6.98e+04 5.35e+11   2.6 6.44e+03    -  4.17e-03 7.19e-04f  5
 266  4.9057523e+09 6.97e+04 9.75e+11   2.6 6.16e+03    -  1.30e-02 6.63e-04h  5
 267  4.9054115e+09 6.97e+04 9.86e+11   2.6 6.15e+03    -  4.88e-03 1.31e-04h  7
 268  4.9052108e+09 6.97e+04 9.67e+11   2.6 6.42e+03    -  1.48e-02 6.21e-05f  9
 269  4.8979259e+09 6.95e+04 4.16e+13   2.6 6.63e+03    -  1.68e-02 2.44e-03h  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 270  4.8264801e+09 6.82e+04 2.04e+13   2.6 6.84e+03    -  3.93e-03 1.91e-02w  1
 271  4.8264733e+09 6.82e+04 2.04e+13   2.6 3.95e+04    -  4.69e-03 3.07e-06w  1
 272  4.8245810e+09 6.82e+04 5.04e+14   2.6 3.66e+04    -  4.42e-03 9.03e-04w  1
 273  4.8956850e+09 6.95e+04

 355  4.2063222e+09 7.77e+04 3.64e+16   3.1 4.90e+06    -  1.37e-02 9.80e-03H  1
 356  4.2063017e+09 7.77e+04 4.41e+17   3.8 4.37e-01  14.2 1.00e+00 1.00e+00h  1
 357  4.2064014e+09 7.77e+04 7.79e+17   2.3 3.90e-01  13.7 1.00e+00 1.00e+00h  1
 358  4.2064549e+09 7.77e+04 3.44e+17   1.0 5.32e-01  13.2 1.00e+00 1.25e-01h  4
 359  4.2065884e+09 7.77e+04 7.97e+17  -0.4 5.14e-01  13.7 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 360  4.2067014e+09 7.77e+04 3.64e+18  -1.5 4.33e-01  14.1 1.00e+00 1.00e+00h  1
 361  4.2066200e+09 7.77e+04 3.64e+18   0.5 1.09e+04    -  1.00e+00 4.31e-05h  1
 362  4.1712007e+09 7.57e+04 7.81e+17   2.5 1.13e+04    -  5.38e-02 6.07e-03f  1
 363  4.1739861e+09 7.57e+04 1.70e+18   3.3 3.04e+04    -  4.82e-04 5.12e-04h  1
 364  4.1739414e+09 7.57e+04 1.70e+18  -4.7 5.76e+06    -  1.27e-06 2.79e-07f  1
 365r 4.1739414e+09 7.57e+04 9.98e+02   3.3 0.00e+00    -  0.00e+00 5.48e-08R  2
 366r 4.1808580e+09 7.47e+04

...

 634r 3.9720331e+09 1.01e+05 6.41e+08   5.0 1.79e+04    -  4.63e-01 4.41e-02f  1
 635r 3.9678525e+09 1.01e+05 2.39e+09   4.7 4.99e+02    -  7.17e-01 5.21e-01f  1
 636r 3.9581460e+09 1.01e+05 4.82e+09   5.0 2.27e+02    -  6.47e-01 8.62e-01f  1
 637r 3.9577130e+09 1.01e+05 3.27e+09   4.9 5.26e+02    -  1.00e+00 3.09e-01f  1
 638r 3.9544954e+09 1.01e+05 2.22e+09   4.9 8.99e+03    -  1.36e-01 1.03e-02f  6
 639r 3.9597028e+09 1.01e+05 8.57e+09   4.9 1.90e+04    -  1.27e-01 8.51e-03h  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 640r 3.9518925e+09 1.01e+05 8.33e+09   4.9 1.05e+04    -  8.67e-02 2.16e-02f  3
 641r 3.9573052e+09 1.01e+05 9.76e+09   4.9 8.56e+04    -  2.76e-02 1.98e-03f  4
 642r 3.9513028e+09 1.01e+05 1.12e+10   4.9 1.23e+04    -  1.21e-01 1.53e-02h  3
 643r 3.9600418e+09 1.01e+05 2.79e+09   4.9 5.13e+03    -  4.24e-02 5.29e-02f  2
 644r 3.9525515e+09 1.01e+05 2.27e+09   4.9 3.39e+04    -  3.44e-02 6.94e-03f  3
 645r 3.9598554e+09 1.01e+05

 727r 3.9743946e+09 1.01e+05 5.24e+14   5.2 6.02e+03    -  1.37e-01 4.68e-02h  4
 728r 3.9726047e+09 1.01e+05 7.43e+14   5.2 3.36e+07    -  7.63e-05 8.29e-06f  3
 729r 3.9711188e+09 1.01e+05 1.63e+15   5.2 1.23e+04    -  1.18e-01 2.36e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 730r 3.9724873e+09 1.01e+05 1.22e+16   5.2 1.79e+03    -  1.62e-01 1.58e-01f  1
 731r 3.9724629e+09 1.01e+05 6.13e+14   5.2 2.63e+03    -  2.27e-01 1.12e-01f  1
 732r 3.9724433e+09 1.01e+05 3.02e+14   5.2 4.71e+03    -  6.27e-01 6.07e-01f  1
 733r 3.9762579e+09 1.01e+05 2.50e+14   5.2 3.40e+04    -  1.47e-01 1.36e-02f  1
 734r 3.9801367e+09 1.01e+05 2.64e+14   5.2 7.48e+03    -  2.99e-01 2.87e-02f  4
 735r 3.9782226e+09 1.01e+05 1.32e+15   5.2 7.12e+03    -  7.82e-01 2.51e-01f  1
 736r 3.9788674e+09 1.01e+05 7.82e+14   5.2 1.40e+03    -  4.21e-01 1.30e-01h  1
 737r 3.9787462e+09 1.01e+05 4.27e+09   5.2 2.91e+03    -  1.59e-01 1.00e+00h  1
 738r 3.9759272e+09 1.01e+05

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 820r 3.9036728e+09 1.01e+05 5.60e+09   4.1 2.28e+02    -  2.64e-01 1.88e-01H  1
 821r 3.9037146e+09 1.01e+05 5.76e+09   4.1 4.48e+02    -  1.30e-01 1.13e-01f  1
 822r 3.9035909e+09 1.01e+05 5.51e+10   4.1 2.98e+02    -  2.69e-01 5.76e-02h  2
 823r 3.8236033e+09 1.01e+05 1.49e+10   4.1 2.98e+03    -  3.48e-01 7.04e-02f  1
 824r 3.8237411e+09 1.01e+05 1.88e+10   4.1 2.15e+02    -  1.05e-01 1.67e-01h  1
 825r 3.8238849e+09 1.01e+05 4.89e+10   4.1 2.77e+02    -  8.78e-02 1.27e-01h  1
 826r 3.8239009e+09 1.01e+05 4.51e+10   4.1 6.93e+02    -  3.86e-02 9.61e-02h  1
 827r 3.8237733e+09 1.01e+05 4.88e+10   4.1 7.77e+02    -  7.30e-02 1.05e-01h  1
 828r 3.8238168e+09 1.01e+05 3.34e+11   4.1 7.72e+02    -  7.62e-02 1.06e-01h  1
 829r 3.8151233e+09 1.01e+05 3.70e+11   4.1 2.00e+04    -  1.86e-01 5.30e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 830r 3.7851264e+09 1.01e+05

 912r 3.0862500e+09 1.00e+05 9.35e+12   4.1 4.60e+02    -  2.18e-01 8.22e-02f  2
 913r 3.0862497e+09 1.00e+05 1.35e+13   4.1 7.74e+02    -  1.10e-01 1.45e-02h  2
 914r 3.0862256e+09 1.00e+05 3.27e+11   4.1 1.79e+02    -  1.55e-01 3.44e-01h  1
 915r 3.0862220e+09 1.00e+05 2.53e+11   4.1 6.67e+02    -  1.58e-01 1.98e-02f  4
 916r 3.0862146e+09 1.00e+05 5.19e+12   4.1 1.18e+03    -  2.30e-01 4.64e-02h  1
 917r 3.0862168e+09 1.00e+05 2.93e+12   4.1 4.18e+02    -  2.53e-01 5.35e-02h  1
 918r 3.0862184e+09 1.00e+05 2.80e+12   4.1 4.77e+02    -  1.86e-01 1.07e-01f  1
 919r 3.0862346e+09 1.00e+05 4.67e+11   4.1 1.73e+02    -  2.19e-01 3.70e-01H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 920r 3.0862371e+09 1.00e+05 1.56e+12   4.1 1.91e+03    -  7.27e-02 2.33e-02f  1
 921r 3.0862343e+09 1.00e+05 5.56e+12   4.1 5.98e+02    -  1.85e-01 3.81e-02f  3
 922r 3.0862262e+09 1.00e+05 1.55e+12   4.1 1.12e+03    -  1.14e-01 4.19e-02f  3
 923r 3.0862276e+09 1.00e+05

[diag] APAP demand residual (scaled) warm-start: 0.000000  (>=0 feasible)
[diag] APAP demand residual (scaled) solution   : 148.063032  (>=0 feasible)
[diag] H2   demand residual (scaled) warm-start: 0.000000   (>=0 feasible)
[diag] H2   demand residual (scaled) solution   : -141.738237   (>=0 feasible)
[diag] D_APAP = 10000, D_H2 = 50000, D_SCALE_APAP = 10000, D_SCALE_H2 = 50000
Solved. Objective J = 3080556717.365986
Wrote opt_states_*.csv, opt_controls_*.csv, opt_buffers.csv
Wrote objective_components_physical.csv
